## Importing libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10 #natural images of (airplanes, animals..)

## Loading dataset

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

## Model

### Sequential api

In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, 3, padding="valid", activation="relu"), # padding valid the px change depending on the kernel size so it becomes 30x30 px
        layers.MaxPooling2D(pool_size=(2, 2)),# the inputs are half the ones before
        layers.Conv2D(64, 3, activation="relu"), # conv layer
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"), # intermediate output layer
        layers.Dense(10), # output layer
    ]
)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    metrics=['accuracy']
)

In [8]:
model.fit(x_train, y_train,batch_size=64,epochs=10,verbose = 2)
model.evaluate(x_test, y_test,batch_size=64,verbose = 2)

Epoch 1/10
782/782 - 35s - 44ms/step - accuracy: 0.3906 - loss: 1.6814
Epoch 2/10
782/782 - 26s - 33ms/step - accuracy: 0.5210 - loss: 1.3443
Epoch 3/10
782/782 - 25s - 32ms/step - accuracy: 0.5747 - loss: 1.2032
Epoch 4/10
782/782 - 25s - 32ms/step - accuracy: 0.6153 - loss: 1.0941
Epoch 5/10
782/782 - 27s - 35ms/step - accuracy: 0.6422 - loss: 1.0196
Epoch 6/10
782/782 - 30s - 39ms/step - accuracy: 0.6642 - loss: 0.9655
Epoch 7/10
782/782 - 27s - 35ms/step - accuracy: 0.6836 - loss: 0.9112
Epoch 8/10
782/782 - 24s - 30ms/step - accuracy: 0.6979 - loss: 0.8678
Epoch 9/10
782/782 - 25s - 32ms/step - accuracy: 0.7132 - loss: 0.8292
Epoch 10/10
782/782 - 30s - 39ms/step - accuracy: 0.7263 - loss: 0.7934
157/157 - 3s - 18ms/step - accuracy: 0.6994 - loss: 0.8764


[0.8764219880104065, 0.699400007724762]

### Functional api

In [9]:
def my_model():
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 5, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [11]:
model = my_model()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

In [12]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 96s - 123ms/step - accuracy: 0.5465 - loss: 1.2735
Epoch 2/10
782/782 - 85s - 108ms/step - accuracy: 0.6938 - loss: 0.8777
Epoch 3/10
782/782 - 66s - 85ms/step - accuracy: 0.7486 - loss: 0.7238
Epoch 4/10
782/782 - 69s - 88ms/step - accuracy: 0.7871 - loss: 0.6142
Epoch 5/10
782/782 - 70s - 89ms/step - accuracy: 0.8153 - loss: 0.5305
Epoch 6/10
782/782 - 66s - 84ms/step - accuracy: 0.8406 - loss: 0.4615
Epoch 7/10
782/782 - 70s - 89ms/step - accuracy: 0.8647 - loss: 0.3934
Epoch 8/10
782/782 - 81s - 103ms/step - accuracy: 0.8860 - loss: 0.3347
Epoch 9/10
782/782 - 62s - 80ms/step - accuracy: 0.9035 - loss: 0.2846
Epoch 10/10
782/782 - 61s - 78ms/step - accuracy: 0.9217 - loss: 0.2353
157/157 - 3s - 18ms/step - accuracy: 0.6997 - loss: 1.0237


[1.0236897468566895, 0.6996999979019165]

Resources : 
- youtube playlist : https://www.youtube.com/playlist?list=PLC28HkOkM1K3drfnM_eetkmD3LGwjoiA_